In [191]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from datetime import datetime
import joblib

In [192]:
RARE_CATEGORY_THRESHOLD = 0.01

CAR_AGE_MIN_THRESHOLD = 0   
CAR_AGE_MAX_THRESHOLD = 100 

MILEAGE_MIN_THRESHOLD = 1000  
MILEAGE_MAX_THRESHOLD = 500000

RAW_DATA_PATH = '../data/raw/UsedCarsSA_Clean_EN.csv'
OUTPUT_DATA_PATH = '../data/processed/procssed.csv'
OUTPUT_PREPROCESSOR_PATH = '../output/models/preprocessor.joblib'

In [193]:
df = pd.read_csv(RAW_DATA_PATH)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8035 entries, 0 to 8034
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Make         8035 non-null   object 
 1   Type         8035 non-null   object 
 2   Year         8035 non-null   int64  
 3   Origin       8035 non-null   object 
 4   Color        8035 non-null   object 
 5   Options      8035 non-null   object 
 6   Engine_Size  8035 non-null   float64
 7   Fuel_Type    8035 non-null   object 
 8   Gear_Type    8035 non-null   object 
 9   Mileage      8035 non-null   int64  
 10  Region       8035 non-null   object 
 11  Price        8035 non-null   int64  
 12  Negotiable   8035 non-null   bool   
dtypes: bool(1), float64(1), int64(3), object(8)
memory usage: 761.3+ KB


In [194]:
if 'Link' in df.columns:
    df = df.drop('Link', axis=1)

In [195]:
df['Price'] = df['Price'].replace('Negotiable', np.nan)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df['Price'] = df['Price'].replace(0, np.nan)
df['Price'].isna().sum()

2527

In [196]:
df.head()

,Make,Type,Year,Origin,Color,Options,Engine_Size,Fuel_Type,Gear_Type,Mileage,Region,Price,Negotiable
0,Chrysler,C300,2018,Saudi,Black,Full,5.7,Gas,Automatic,103000,Riyadh,114000.0,False
1,Nissan,Patrol,2016,Saudi,White,Full,4.8,Gas,Automatic,5448,Riyadh,NaN,True
2,Nissan,Sunny,2019,Saudi,Silver,Standard,1.5,Gas,Automatic,72418,Riyadh,27500.0,False
3,Hyundai,Elantra,2019,Saudi,Grey,Standard,1.6,Gas,Automatic,114154,Riyadh,43000.0,False
4,Hyundai,Elantra,2019,Saudi,Silver,Semi Full,2.0,Gas,Automatic,41912,Riyadh,59500.0,False


In [197]:
df['Negotiable'] = df['Negotiable'].astype(int)

In [198]:
df['Mileage'] = df['Mileage'].apply(lambda x: x if MILEAGE_MIN_THRESHOLD <= x <= MILEAGE_MAX_THRESHOLD else np.nan)
df['Mileage'].isna().sum()

1059

In [199]:
current_year = datetime.now().year
df['Car_Age'] = current_year - df['Year']
df['mileage_par_age'] = df['Mileage'] / df['Car_Age'].replace(0,1)
df = df.drop('Year', axis=1)
dr = not 'Year' in df.columns
print(f"""
      Car_Age None : {df['Car_Age'].isna().sum()}
      Droped Year : {dr}
      """)



      Car_Age None : 0
      Droped Year : True
      


In [200]:
numerical_cols_indemnity = []
categorical_cols_indemnity = []
for col in df.columns:
    dr = df[col].isna().sum()
    if dr > 0 :
        if pd.api.types.is_numeric_dtype(df[col]):
            numerical_cols_indemnity.append(col)
            print(f"{col} None: | {dr} | type raw : Number")
        else:
            categorical_cols_indemnity.append(col)
            print(f"{col} None: | {dr} | type raw : String")

Mileage None: | 1059 | type raw : Number
Price None: | 2527 | type raw : Number
mileage_par_age None: | 1059 | type raw : Number


In [201]:
for col in numerical_cols_indemnity:
    if col in df.columns:
        df[col].fillna(df[col].median(), inplace=True)

In [202]:
def winsorize_column(series, lower_percentile=0.01, upper_percentile=0.99):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    IR = q3 - q1
    lower_bound = series.quantile(lower_percentile)
    upper_bound = series.quantile(upper_percentile)
    if ((series < (q1-1.5*IR)) | (series > (q3+1.5*IR))).any():
        return series.clip(lower=lower_bound, upper=upper_bound)
    return series

In [203]:
numerical_cols = []
categorical_cols = []
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        numerical_cols.append(col)
        print(f"{col} Type: | {df[col].dtype} | type raw : Number")
    else:
        categorical_cols.append(col)
        print(f"{col} Type: | {df[col].dtype} | type raw : String")

Make Type: | object | type raw : String
Type Type: | object | type raw : String
Origin Type: | object | type raw : String
Color Type: | object | type raw : String
Options Type: | object | type raw : String
Engine_Size Type: | float64 | type raw : Number
Fuel_Type Type: | object | type raw : String
Gear_Type Type: | object | type raw : String
Mileage Type: | float64 | type raw : Number
Region Type: | object | type raw : String
Price Type: | float64 | type raw : Number
Negotiable Type: | int32 | type raw : Number
Car_Age Type: | int64 | type raw : Number
mileage_par_age Type: | float64 | type raw : Number


In [204]:
for col in ['Price','Mileage']:
    df[col] = winsorize_column(df[col])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8035 entries, 0 to 8034
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Make             8035 non-null   object 
 1   Type             8035 non-null   object 
 2   Origin           8035 non-null   object 
 3   Color            8035 non-null   object 
 4   Options          8035 non-null   object 
 5   Engine_Size      8035 non-null   float64
 6   Fuel_Type        8035 non-null   object 
 7   Gear_Type        8035 non-null   object 
 8   Mileage          8035 non-null   float64
 9   Region           8035 non-null   object 
 10  Price            8035 non-null   float64
 11  Negotiable       8035 non-null   int32  
 12  Car_Age          8035 non-null   int64  
 13  mileage_par_age  8035 non-null   float64
dtypes: float64(4), int32(1), int64(1), object(8)
memory usage: 847.6+ KB


In [205]:
def group_rare_categories(df, column, threshold):
    counts = df[column].value_counts(normalize=True)
    rare_categories = counts[counts < threshold].index
    df[column] = df[column].replace(rare_categories, 'Rare')
    return df

In [206]:
for col in ['Type','Region']:
    df = group_rare_categories(df,col,RARE_CATEGORY_THRESHOLD)

In [207]:
numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
numerical_cols.remove('Price')
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

In [208]:
numerical_cols

['Engine_Size', 'Mileage', 'Negotiable', 'Car_Age', 'mileage_par_age']

In [209]:
X = df.drop('Price', axis=1)
y = df['Price']
X_processed = preprocessor.fit_transform(X)

In [210]:
all_cols_names = numerical_cols + list(preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_cols))
df_processed = pd.DataFrame(X_processed.toarray(), columns=all_cols_names)
df_processed['Price'] = y.values
df_processed.to_csv(OUTPUT_DATA_PATH, index=False)
joblib.dump(preprocessor,OUTPUT_PREPROCESSOR_PATH)

['../output/models/preprocessor.joblib']